# Examples of running HypoInvPy with utility functions
The examples use the phase data from running EQTransformer. 

In [11]:
import os, sys,glob
import pandas as pd
import numpy as np
from hypoinvpy import core as hc
from hypoinvpy import utils
import matplotlib.pyplot as plt

## Convert station information from json database to CSV
The example here uses the *.json file saved by EQTransforer downloading step. The CSV file is for general purpose. To reformat it for HypoInverse code, please see the next example.

In [ ]:
infile='input/EQT_station_list.json'
utils.stainfo_json2csv(infile,outfile='input/EQT_station_list.csv')
stainfo=utils.stainfo_json2csv(infile)
print(stainfo)

## Reformat station information for HypoInverse code.
The `HypoInvPy.core.reformat_stainfo()` function takes input in `json` or `csv` format. See the source code or use `help()` to get detailed usage information of this function. 

* `force_channel_type` option is to force all channels to the specified type. This is because in some earthquake detection and association steps, the original channel types might be lost, making it impossible to link the phase data with station information.
* `rename_component_dict` option is to rename component names. This is needec when some components originally having 1 or 2 in the names. These original names sometimes are not kept through the earthquake detection and association steps.

In [ ]:
hc.reformat_stainfo(infile,'input/EQT_station_list.sta',informat='json',force_channel_type='HH',
                   rename_component_dict={'1':'N','2':'E'})

## Setup parameters for HypoInverse code
Major parameters to run the code are carried through with a container class named `HypoInvPy.core.HypoInvConfig()`. This class is simplified from the `config()` class in `Hypo-Interface-Py`. In `HypoInvPy`, I also reduced the number of metadata/parameters, to focus only on required key parameters.

Examples here use the phase information directly from the `EQTransformer` association step. The station information file has been reformated as shown in the previous cell. 

In [4]:
# modified from https://github.com/YijianZhou/Hypo-Interface-Py
phase_file='input/EQT_Y2000.phs'
station_file='input/EQT_station_list.sta'
pmodel='input/velo_p_eg.cre'
smodel='input/velo_s_eg.cre'
lat_code='N'
lon_code='W'
depth_try_list=np.arange(0,20,1)
min_nsta=4 #minimum number of stations to relocate the earthquake. 4 is recommeneded as the minimum to get a reliable location.
keep_summary_files=False #keep intermediate summary files from each relocation run?
outputdir='output'

cfg = hc.HypoInvConfig(phase_file=phase_file,station_file=station_file,
                       pmodel=pmodel,smodel=smodel,min_nsta=min_nsta,
                      lat_code=lat_code,lon_code=lon_code,ztrlist=depth_try_list)
#file names for the final summary catalogs.
out_bad = '%s/%s_bad.csv'%(outputdir,cfg.run_tag)
out_good = '%s/%s_good.csv'%(outputdir,cfg.run_tag)

## Run hypoinverse through the interface
This is the key step to run hypoinverse. I kept the way Hypo-Interface-Py uses running the code on a series of initial depths. This list is specified as `ztrlist` in the configuration step above.

In [ ]:
# 1. produce parameter files for each testing initial depth.
hypparfiledir='input'
hypparfilelist=hc.generate_parfile(cfg,hypparfiledir,outdir=outputdir)

# 2. Run hypoinverse with all parameter files.
hc.run_hypoinv(hypparfilelist)

## Extract inversion results based on quality

In [ ]:
print('converting output sum files')
summary_filelist=glob.glob('%s/%s-*.sum'%(outputdir,cfg.run_tag))

hc.merge_summary(summary_filelist,out_good,out_bad,lat_code,lon_code)


In [8]:
# remove intermidiate files
for fname in glob.glob('fort.*'): os.unlink(fname)
for fname in glob.glob('input/%s-*.hyp'%cfg.run_tag): os.unlink(fname)
if not keep_summary_files:
    for fname in glob.glob(outputdir+'/'+cfg.run_tag+'*.sum'): os.unlink(fname)

## Read and plot the earthquakes

In [ ]:
quakes=pd.read_csv(out_good,header=None,names=["datetime","latitude","longitude","depth","magnitude","evid"])
print(quakes)

In [ ]:
plt.scatter(quakes.longitude,quakes.latitude,20*np.power(3,quakes.magnitude),quakes.depth,edgecolors='k')
plt.colorbar(label='depth (m)')
plt.show()